# Data Preprocessing and Cleaning
## USDA FoodData Central - Branded Foods Dataset

This notebook handles data cleaning, preprocessing, and preparation for analysis including:
- Data loading and initial assessment
- Missing value treatment
- Data type conversions and standardization
- Text preprocessing for ingredients
- Data quality validation
- Export of cleaned datasets

Input: Raw CSV files from DATA directory
Output: Cleaned datasets saved to RESULTS directory

In [1]:
import pandas as pd
import numpy as np
import re
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Create results directories
results_dir = Path('../RESULTS')
data_dir = results_dir / 'processed_data'
figures_dir = results_dir / 'figures'
models_dir = results_dir / 'models'

for directory in [results_dir, data_dir, figures_dir, models_dir]:
    directory.mkdir(exist_ok=True)

print("Libraries loaded and directories created")

Libraries loaded and directories created


## Data Loading and Initial Assessment

In [2]:
# Load main datasets
print("Loading datasets...")
branded_df = pd.read_csv("../DATA/branded_food.csv")
nutrient_df = pd.read_csv("../DATA/food_nutrient.csv")
nutrient_ref = pd.read_csv("../DATA/nutrient.csv")
food_df = pd.read_csv("../DATA/food.csv")

print(f"Branded Foods Dataset: {branded_df.shape}")
print(f"Food Nutrients Dataset: {nutrient_df.shape}")
print(f"Nutrient Reference Dataset: {nutrient_ref.shape}")
print(f"Food Reference Dataset: {food_df.shape}")

# Initial data quality assessment
print("\nData Quality Assessment:")
for name, df in [("Branded Foods", branded_df), ("Food Nutrients", nutrient_df)]:
    missing_pct = (df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100
    print(f"{name}: {missing_pct:.1f}% missing values")

Loading datasets...


MemoryError: Unable to allocate 1.53 GiB for an array with shape (8, 25652681) and data type float64

## Text Preprocessing for Ingredients

In [ ]:
def clean_ingredients_text(text):
    """Clean and standardize ingredient text"""
    if not isinstance(text, str) or pd.isna(text):
        return ""
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove common prefixes
    text = re.sub(r"^\s*ingredients?\s*:\s*", "", text)
    text = re.sub(r"^\s*contains\s*:\s*", "", text)
    
    # Handle "contains less than X%" phrases
    text = re.sub(
        r"\b(?:contains\s+less\s+than\s*\d+%|" +
        r"contains\s*\d+%\s*or\s*less|" +
        r"\d+%\s*or\s*less|" +
        r"less\s+than\s*\d+%)\s*(?:of)?\s*:?\s*", 
        "", text
    )
    
    # Remove parenthetical content (processing details)
    while True:
        new_text = re.sub(r"\([^()]*\)", "", text)
        if new_text == text:
            break
        text = new_text
    
    # Standardize separators
    text = re.sub(r"\band/or\b", ",", text)
    text = text.replace(";", ",")
    
    # Clean whitespace and commas
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"\s*,\s*", ", ", text)
    text = re.sub(r"(,\s*){2,}", ", ", text)
    text = text.strip(" ,")
    
    return text

def parse_ingredients_to_list(text):
    """Convert ingredient text to standardized list"""
    if not isinstance(text, str) or not text.strip():
        return []
    
    # Split by commas and clean each ingredient
    ingredients = [ing.strip() for ing in text.split(',')]
    ingredients = [ing for ing in ingredients if ing and len(ing) > 1]
    
    return ingredients

# Apply cleaning to branded foods
print("Cleaning ingredient text...")
branded_df['ingredients_clean'] = branded_df['ingredients'].apply(clean_ingredients_text)
branded_df['ingredients_list'] = branded_df['ingredients_clean'].apply(parse_ingredients_to_list)
branded_df['ingredient_count'] = branded_df['ingredients_list'].apply(len)

print(f"Processed {len(branded_df)} products")
print(f"Products with ingredients: {(branded_df['ingredient_count'] > 0).sum()}")
print(f"Average ingredients per product: {branded_df['ingredient_count'].mean():.1f}")

## Date Processing and Standardization

In [ ]:
# Process date columns
date_columns = ['modified_date', 'available_date', 'discontinued_date']

for col in date_columns:
    if col in branded_df.columns:
        branded_df[col] = pd.to_datetime(branded_df[col], errors='coerce')
        print(f"Processed {col}: {branded_df[col].notna().sum()} valid dates")

# Extract temporal features
if 'available_date' in branded_df.columns:
    branded_df['year_available'] = branded_df['available_date'].dt.year
    branded_df['month_available'] = branded_df['available_date'].dt.month
    branded_df['is_recent'] = (branded_df['year_available'] >= 2020).astype(int)
    
    print(f"Date range: {branded_df['available_date'].min()} to {branded_df['available_date'].max()}")

## Data Type Optimization and Missing Value Treatment

In [ ]:
# Optimize data types for memory efficiency
def optimize_dtypes(df):
    """Optimize dataframe dtypes for memory efficiency"""
    original_memory = df.memory_usage(deep=True).sum() / 1024**2
    
    # Convert object columns with few unique values to category
    for col in df.select_dtypes(include=['object']).columns:
        if col not in ['ingredients', 'ingredients_clean', 'ingredients_list']:
            unique_ratio = df[col].nunique() / len(df)
            if unique_ratio < 0.1:  # Less than 10% unique values
                df[col] = df[col].astype('category')
    
    # Convert numeric columns
    for col in df.select_dtypes(include=['int64']).columns:
        if df[col].min() >= 0:
            if df[col].max() < 255:
                df[col] = df[col].astype('uint8')
            elif df[col].max() < 65535:
                df[col] = df[col].astype('uint16')
            else:
                df[col] = df[col].astype('uint32')
    
    optimized_memory = df.memory_usage(deep=True).sum() / 1024**2
    print(f"Memory usage: {original_memory:.1f}MB -> {optimized_memory:.1f}MB ({optimized_memory/original_memory:.1%})")
    
    return df

# Apply optimization
print("Optimizing branded foods dataset...")
branded_df = optimize_dtypes(branded_df)

print("\nOptimizing nutrients dataset...")
nutrient_df = optimize_dtypes(nutrient_df)

## Data Quality Validation

In [ ]:
# Validate data quality
def validate_data_quality(df, name):
    """Perform data quality checks"""
    print(f"\nData Quality Report: {name}")
    print("=" * 40)
    
    # Duplicate check (exclude list columns) - sample for large datasets
    hashable_columns = []
    for col in df.columns:
        try:
            # Test if column can be hashed (exclude list columns)
            sample_val = df[col].iloc[0] if len(df) > 0 else None
            if not isinstance(sample_val, list):
                hashable_columns.append(col)
        except (TypeError, IndexError):
            continue
    
    if hashable_columns:
        # For large datasets, use sampling for duplicate check
        if len(df) > 1_000_000:
            sample_size = min(100_000, len(df))
            sample_df = df[hashable_columns].sample(n=sample_size, random_state=42)
            duplicates_in_sample = sample_df.duplicated().sum()
            duplicate_rate = duplicates_in_sample / sample_size
            estimated_duplicates = int(duplicate_rate * len(df))
            print(f"Estimated duplicate rows (from {sample_size:,} sample): ~{estimated_duplicates:,} ({duplicate_rate:.2%})")
        else:
            duplicates = df[hashable_columns].duplicated().sum()
            print(f"Duplicate rows (excluding list columns): {duplicates}")
    else:
        print("Cannot check duplicates - no hashable columns")
    
    # Missing value summary
    missing_summary = df.isnull().sum()
    missing_cols = missing_summary[missing_summary > 0]
    print(f"Columns with missing values: {len(missing_cols)}")
    
    if len(missing_cols) > 0:
        print("\nTop missing value columns:")
        for col, count in missing_cols.head().items():
            pct = (count / len(df)) * 100
            print(f"  {col}: {count} ({pct:.1f}%)")
    
    # Data completeness score
    completeness = (df.notna().sum().sum() / (len(df) * len(df.columns))) * 100
    print(f"\nOverall completeness: {completeness:.1f}%")
    
    return completeness

# Validate datasets
branded_completeness = validate_data_quality(branded_df, "Branded Foods")
nutrient_completeness = validate_data_quality(nutrient_df, "Food Nutrients")

# Additional validation for branded foods
print("\nBranded Foods Specific Validation:")
print(f"Products with valid FDC IDs: {branded_df['fdc_id'].notna().sum()}")
print(f"Products with brand information: {branded_df['brand_owner'].notna().sum()}")
print(f"Products with ingredients: {(branded_df['ingredient_count'] > 0).sum()}")
print(f"Products with categories: {branded_df['branded_food_category'].notna().sum()}")

## Export Cleaned Datasets

In [ ]:
# Export cleaned datasets
print("Exporting cleaned datasets...")

# Export main cleaned dataset
output_path = data_dir / 'branded_food_cleaned.csv'
branded_df.to_csv(output_path, index=False)
print(f"Branded foods saved: {output_path}")

# Export optimized nutrients dataset
nutrient_output_path = data_dir / 'food_nutrient_cleaned.csv'
nutrient_df.to_csv(nutrient_output_path, index=False)
print(f"Food nutrients saved: {nutrient_output_path}")

# Export nutrient reference
nutrient_ref_path = data_dir / 'nutrient_reference.csv'
nutrient_ref.to_csv(nutrient_ref_path, index=False)
print(f"Nutrient reference saved: {nutrient_ref_path}")

# Create processing summary
summary = {
    'processing_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'original_branded_records': len(branded_df),
    'branded_completeness_pct': branded_completeness,
    'products_with_ingredients': (branded_df['ingredient_count'] > 0).sum(),
    'avg_ingredients_per_product': branded_df['ingredient_count'].mean(),
    'nutrient_records': len(nutrient_df),
    'nutrient_completeness_pct': nutrient_completeness,
    'date_range_start': str(branded_df['available_date'].min()) if 'available_date' in branded_df.columns else 'N/A',
    'date_range_end': str(branded_df['available_date'].max()) if 'available_date' in branded_df.columns else 'N/A'
}

summary_path = data_dir / 'preprocessing_summary.json'
import json
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2, default=str)

print(f"\nProcessing summary saved: {summary_path}")
print("\nPreprocessing complete! Cleaned datasets are ready for feature engineering.")

## Preprocessing Summary

### Completed Tasks:
1. **Data Loading**: Loaded all USDA FoodData Central files
2. **Text Preprocessing**: Cleaned and standardized ingredient text
3. **Date Processing**: Converted date columns and extracted temporal features
4. **Data Optimization**: Optimized data types for memory efficiency
5. **Quality Validation**: Performed comprehensive data quality checks
6. **Export**: Saved cleaned datasets to RESULTS/processed_data/

### Data Quality Improvements:
- Standardized ingredient text formatting
- Parsed ingredients into structured lists
- Optimized memory usage through appropriate data types
- Extracted temporal features from dates
- Created comprehensive quality metrics

### Output Files:
- `branded_food_cleaned.csv`: Main cleaned dataset
- `food_nutrient_cleaned.csv`: Cleaned nutritional data
- `nutrient_reference.csv`: Nutrient reference table
- `preprocessing_summary.json`: Processing metadata

Next: Feature Engineering (03_Feature_Engineering.ipynb)